In [1]:
import pandas as pd

import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

from fuzzywuzzy import process

In [5]:
#Elecion data
candidate_demo_df = pd.read_csv('../../cc_election_cleaning/electoral_clustering_with_pca.csv')
all_candidate_demo_df = pd.read_csv('../../cc_election_cleaning/all_candidate_electoral_clusters.csv')
weighted_demographics = pd.read_csv('../../cc_election_cleaning/weighted_demographics_clusters.csv')
#Finance data
winning_finance_cluster = pd.read_csv('../../council_campaign_finance/data/winning_members_clusters.csv')
all_finance_cluster = pd.read_csv('../../council_campaign_finance/data/all_members_clusters.csv')
#Sponsorship data
sponsorship_df = pd.read_csv('../../cc_legislation/data/cosponsorship_matrix_clusters.csv')

In [ ]:
# clusters and meta cluster from voting and sponsorship
meta_clusters = pd.read_csv('../../cc_legislation/data/meta_clusters.csv')